# Multi terrain probabistic sensing

# About this file

This file is to use the functionalities of the classes defined in the other file.

## Things to be done here

1)

## Things done here

1)

In [3]:
%run a_star_2.ipynb
import numpy as np
%run agent_6.ipynb

import random

In [4]:
maze_size = 10
probability = 0.3

# Code for Agent 6 goes here

In [5]:
def run_agent_6(agent6, maze):
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent6.agent_gridworld

    target_reached = False
    while not target_reached:
        current_target = agent6.getMaxProbabilityCell(source)
        print('Temporarily checking:',current_target)
        _,_,_,path = astar(source,current_target,  agent6.agent_gridworld)
        print(path)
        for node in path:
            if full_grid_world_matrix.item(node) == 0:
                terrain_type = terrain_matrix.item(node)
                agent6.update_probabilities(node, 'open',  terrain_type)
                break
            elif full_grid_world_matrix.item(node) == 1:
                print('cell blocked, replanning route')
                agent6.update_probabilities(node,'blocked')
                agent6.agent_gridworld.itemset(node,1)
                run_agent_6(agent6, maze)
#                 _,_,_,path = astar(source,current_target,  agent6.agent_gridworld)

        if path:
            target_reached = agent6.examine(maze, path[-1])
        else:
            target_reached = False
        current_target = source

        if (not target_reached):
            agent6.update_probabilities(node, 'open',  terrain_type)


# Main function

In [ ]:
if __name__ == "__main__":

#     source,target=set_valid_source_target()
    maze = gridworld(maze_size, probability)
    agent6 = agent_6(maze_size)
    run_agent_6(agent6, maze)

In [ ]:
plt.spy(maze.full_grid_world_matrix)